In [7]:
# Contiene información técnica y de mercado de automóviles. El objetivo es predecir 
# el precio del automóvil en función de sus atributos


# Para entrenar un modelo de regresión lineal y configurar diferentes hiperparámetros, puedes utilizar el método GridSearchCV

# Importar librerías necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression

# Cargar el dataset
data = pd.read_csv('Dataset_Regresion_Lineal/Car details v3.csv')

# Limpieza de datos
columns_to_drop = ['name']  # Eliminar columnas irrelevantes
data = data.drop(columns=columns_to_drop)
data = data.dropna()  # Eliminar filas con valores nulos

# Convertir variables categóricas a variables numéricas con One Hot Encoding
data = pd.get_dummies(data, drop_first=True)

# Separar características y el target (Precio)
X = data.drop('selling_price', axis=1)
y = data['selling_price']

# Escalar las características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Selección de características
selector = SelectKBest(score_func=f_regression, k=10)  # Ajusta 'k' según sea necesario
X_selected = selector.fit_transform(X_scaled, y)

# Dividir el dataset en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Configuración de hiperparámetros para GridSearchCV
# Definir el modelo (puedes probar con LinearRegression, Ridge o Lasso)
model = Ridge()  # Cambia a Lasso() si lo prefieres

# Definir el rango de hiperparámetros
param_grid = {
    'alpha': [0.1, 1, 10, 100],  # Solo para Ridge o Lasso
    'fit_intercept': [True, False]  # Ajustar el término de intercepción
}

# Configurar GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros
best_params = grid_search.best_params_
print(f'Los mejores hiperparámetros son: {best_params}')

# Predecir en el conjunto de prueba usando el mejor modelo
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Evaluar el modelo
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'\nMean Squared Error (MSE): {mse}')
print(f'R² Score: {r2}')



Los mejores hiperparámetros son: {'alpha': 10, 'fit_intercept': True}

Mean Squared Error (MSE): 211303023964.91312
R² Score: 0.6960906398651248
